In [1]:
import pandas as pd
from numpy import ceil
from tqdm import tqdm

file = 'E:/playrix.csv'
n_rows = 147893968

### В этом ноутбуке было сделано следующее:
* создание справочника уникальных uid для того чтобы их проиндексировать
* разбиение основного файла на части, замена uid на индекс user_id
* деление event_time на 1000
* соединение в один файл

In [2]:
def split_for_ranges(number, k):
    rng = int(ceil(number // k) + 1)
    return [k, rng]

In [3]:
for_loop = split_for_ranges(n_rows, 10)
for_loop

[10, 14789397]

In [ ]:
# таблица для уникальных uid
unique_uid = pd.read_csv(file, sep = ';', compression = 'gzip', usecols = [0], skiprows = 0, nrows = 0)

for i in range(0, for_loop[0]):
    df_part = pd.read_csv(file, sep = ';', usecols = [0], 
                      skiprows = 1 + i * for_loop[1], nrows = for_loop[1], names = ['uid'])
    unique_uid = pd.concat([unique_uid, df_part], ignore_index=True)
    unique_uid.drop_duplicates(inplace = True)

In [5]:
unique_uid.head()

,user_id,uid
0,0,c10463f4677393cba48cbc835d89edf6
1,1,cfb9547f099ff6a6ac802423ba6ea7f6
2,2,2f80fdf03f7db2ba13ef66f02ecc5c4f
3,3,5e703287f3fee82504da8912605864b5
4,4,7fdd02ba686aab3864e305b33a190215


In [ ]:
unique_uid.to_csv('unique_uid.csv')

In [4]:
unique_uid = pd.read_csv('unique_uid.csv')
unique_uid.columns = ['user_id', 'uid']

action = {'f':0, 'c':1}

### Разбиение csv файла на несколько + обработка

In [ ]:
for i in tqdm(range(0, for_loop[0])):
    df_part = pd.read_csv(file, sep = ';',
                      skiprows = 1 + i * for_loop[1], nrows = for_loop[1], names = ['uid', 'action', 'level','event_time'])
    
    df_part = pd.merge(left = df_part, right = unique_uid, how = 'left', on = 'uid')[['user_id', 'action', 'level', 'event_time']]
    df_part.replace(to_replace = {'action': action}, inplace = True)
    
    df_part['event_time'] = df_part['event_time'] / 1000
    df_part['event_time'] = df_part['event_time'].astype('int')

    df_part.to_csv('clear_df_'+str(i)+'.csv')

### Объединение файлов в один

In [ ]:
df = pd.read_csv('clear_df_0.csv', usecols = [1,2,3,4])
for i in tqdm(range(1, for_loop[0])):
    df_part = pd.read_csv('clear_df_'+str(i)+'.csv', usecols = [1,2,3,4])
    df = pd.concat([df, df_part], ignore_index=True)
df.to_csv('final_df.csv', index = False)

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [30:33<00:00, 347.11s/it]
